In [1]:
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras import models, regularizers
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
cifar10 = tf.keras.datasets.cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

num_train = int(x_train.shape[0] * 0.8)
num_val = x_train.shape[0] - num_train
mask = list(range(num_train, num_train+num_val))
x_val = x_train[mask]
y_val = y_train[mask]

mask = list(range(num_train))
x_train = x_train[mask]
y_train = y_train[mask]
print(x_train.shape)

(40000, 32, 32, 3)


In [0]:
def conv2d_bn_relu(x, filters, kernel_size, name, weight_decay=.0, strides=(1, 1), use_bn=True):
    conv_name = name + "-conv"
    x = Conv2D(filters=filters,
               kernel_size=kernel_size,
               strides=strides,
               padding='same',
               use_bias=False,
               kernel_regularizer=regularizers.l2(weight_decay),
               name=conv_name)(x)
    if use_bn:
        bn_name = name + "-bn"
        x = BatchNormalization(scale=False, axis=3, name=bn_name)(x)
    relu_name = name + "-relu"
    x = Activation('relu', name=relu_name)(x)
    return x

In [0]:
def inception_block(x, filters_num_array, name, weight_decay=.0, use_bn=True):
    (br0_filters, br1_filters, br2_filters, br3_filters) = filters_num_array
    # br0
    br0 = conv2d_bn_relu(x,
                         filters=br0_filters, kernel_size=(1, 1), weight_decay=weight_decay,
                         name=name + '-br0-1x1', use_bn=use_bn)

    # br1
    br1 = conv2d_bn_relu(x,
                         filters=br1_filters[0], kernel_size=(1, 1), weight_decay=weight_decay,
                         name=name + '-br1-1x1', use_bn=use_bn)
    br1 = conv2d_bn_relu(br1,
                         filters=br1_filters[1], kernel_size=(3, 3), weight_decay=weight_decay,
                         name=name + '-br1-3x3', use_bn=use_bn)

    # br2
    br2 = conv2d_bn_relu(x,
                         filters=br2_filters[0], kernel_size=(1, 1), weight_decay=weight_decay,
                         name=name + '-br2-1x1', use_bn=use_bn)
    br2 = conv2d_bn_relu(br2, filters=br2_filters[1], kernel_size=(5, 5), name=name + '-br2-5x5', use_bn=use_bn)

    # br3
    br3 = MaxPooling2D(pool_size=(3, 3), strides=(1, 1), padding='same', name=name + '-br3-pool')(x)
    br3 = conv2d_bn_relu(br3, filters=br3_filters, kernel_size=(1, 1), weight_decay=weight_decay, name=name + '-br3-1x1')

    x = concatenate([br0, br1, br2, br3], axis=3, name=name)
    return x

In [0]:
def Inception_V1(input_shape, classes, weight_decay=.0, use_bn=True):
    input = Input(shape=input_shape)
    x = input

    x = conv2d_bn_relu(x,
                       filters=64, kernel_size=(1, 1), weight_decay=weight_decay,
                       name='2a', use_bn=use_bn)
    x = conv2d_bn_relu(x, filters=192, kernel_size=(3, 3), weight_decay=weight_decay,
                       name='2b', use_bn=use_bn)

    # inception3a
    x = inception_block(x, (64, (96, 128), (16, 32), 32),
                           weight_decay=weight_decay,
                           name='inception3a', use_bn=use_bn)
    # inception3b
    x = inception_block(x, (128, (128, 192), (32, 96), 64),
                           weight_decay=weight_decay,
                           name='inception3b', use_bn=use_bn)
    x = MaxPool2D(pool_size=(3, 3), strides=(2, 2), padding='same', name='3pool')(x)

    # inception4a
    x = inception_block(x, (192, (96, 208), (16, 48), 64),
                           weight_decay=weight_decay,
                           name='inception4a', use_bn=use_bn)
    # inception4b
    x = inception_block(x, (160, (112, 224), (24, 64), 64),
                           weight_decay=weight_decay,
                           name='inception4b', use_bn=True)
    # inception4c
    x = inception_block(x, (128, (128, 256), (24, 64), 64),
                           weight_decay=weight_decay,
                           name='inception4c', use_bn=use_bn)
    # inception4d
    x = inception_block(x, (112, (144, 288), (32, 64), 64),
                           weight_decay=weight_decay,
                           name='inception4d', use_bn=use_bn)
    # inception4e
    x = inception_block(x, (256, (160, 320), (32, 128), 128),
                           weight_decay=weight_decay,
                           name='inception4e', use_bn=use_bn)
    x = MaxPool2D(pool_size=(3, 3), strides=(2, 2), padding='same', name='4pool')(x)

    # inception5a
    x = inception_block(x, (256, (160, 320), (32, 128), 128),
                           weight_decay=weight_decay,
                           name='inception5a', use_bn=use_bn)
    # inception5b
    x = inception_block(x, (384, (192, 384), (48, 128), 128),
                           weight_decay=weight_decay,
                           name='inception5b', use_bn=use_bn)

    # average pool
    x = AveragePooling2D(pool_size=(8, 8), strides=(1, 1), padding='valid', name='avg8x8')(x)
    x = Flatten(name='flatten')(x)
    x = Dense(classes, activation='softmax', name='predictions')(x)
    model = models.Model(input, x, name='inception_v1')
    return model

In [6]:
weight_decay = 5e-4
lr = 1e-1
num_classes = 10
model = Inception_V1(input_shape=(32, 32, 3), classes=num_classes, weight_decay=weight_decay)
sgd = tf.keras.optimizers.SGD(lr=lr, momentum=0.9, nesterov=False)
model.compile(optimizer=sgd,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


In [7]:
datagen = ImageDataGenerator(horizontal_flip=True, 
                             width_shift_range=0.1, 
                             height_shift_range=0.1)

datagen.fit(x_train)

model.fit_generator(datagen.flow(x_train, y_train, batch_size=128),
                    steps_per_epoch=x_train.shape[0] // 128,
                    epochs=30,
                    verbose=1,
                    validation_data=(x_val, y_val))

Epoch 1/30
313/313 [==============================] - 352s 1s/step - loss: 5.7111 - acc: 0.2729 - val_loss: 11.5488 - val_acc: 0.1727
Epoch 2/30
313/313 [==============================] - 337s 1s/step - loss: 3.8446 - acc: 0.3994 - val_loss: 4.6060 - val_acc: 0.1253
Epoch 3/30
313/313 [==============================] - 338s 1s/step - loss: 2.5931 - acc: 0.5078 - val_loss: 3.8104 - val_acc: 0.2277
Epoch 4/30
313/313 [==============================] - 338s 1s/step - loss: 1.8874 - acc: 0.5939 - val_loss: 3.7822 - val_acc: 0.2364
Epoch 5/30
313/313 [==============================] - 338s 1s/step - loss: 1.5060 - acc: 0.6482 - val_loss: 2.3693 - val_acc: 0.4077
Epoch 6/30
313/313 [==============================] - 338s 1s/step - loss: 1.2610 - acc: 0.6963 - val_loss: 1.9857 - val_acc: 0.4938
Epoch 7/30
313/313 [==============================] - 339s 1s/step - loss: 1.1005 - acc: 0.7332 - val_loss: 2.3325 - val_acc: 0.4083
Epoch 8/30
313/313 [==============================] - 338s 1s/step -

In [8]:
model.evaluate(x_test, y_test)

10000/10000 [==============================] - 30s 3ms/sample - loss: 1.4598 - acc: 0.6449


[1.4598083019256591, 0.6449]